In [1]:
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [2]:
n_gpu_layers = 5  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [3]:
a = [0.2, 0.3, 0.6, 0.7, 0.5],
b = [10.2, 10.3, 10.6, 10.7, 10.5]

In [4]:
import numpy as np

In [5]:
np.var(a)

0.0344

In [6]:
np.var(b)

0.034399999999999945

In [7]:
_PATH_LLAMA_ = "/Users/niteshkumarsharma/.cache/lm-studio/models/TheBloke/Llama-2-7b-Chat-GGUF/llama-2-7b-chat.Q5_K_M.gguf"
_PATH_MISTRAL_ = "/Users/niteshkumarsharma/.cache/lm-studio/models/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/mistral-7b-instruct-v0.1.Q5_K_M.gguf"
_PATH_ORCA_MISTRAL_ = "/Users/niteshkumarsharma/.cache/lm-studio/models/TheBloke/Mistral-7B-OpenOrca-GGUF/mistral-7b-openorca.Q5_K_M.gguf"

In [8]:
MISTRAL_INS = LlamaCpp(model_path=_PATH_MISTRAL_,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=4096,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,
    max_tokens=2000)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/niteshkumarsharma/.cache/lm-studio/models/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/mistral-7b-instruct-v0.1.Q5_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q5_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q5_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q5_K     [  4096, 14336,     1,     1 ]
llama_mode

In [9]:
LLAMA = LlamaCpp(
    model_path=_PATH_LLAMA_,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=4096,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,
    max_tokens=2000)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /Users/niteshkumarsharma/.cache/lm-studio/models/TheBloke/Llama-2-7b-Chat-GGUF/llama-2-7b-chat.Q5_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor

In [10]:
MISTRAL_ORCA = LlamaCpp(
    model_path=_PATH_ORCA_MISTRAL_,
    n_gpu_layers=n_gpu_layers,
    n_batch=64,
    n_ctx=4096,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,
    max_tokens=2000)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/niteshkumarsharma/.cache/lm-studio/models/TheBloke/Mistral-7B-OpenOrca-GGUF/mistral-7b-openorca.Q5_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  4096, 32002,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q5_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q5_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q5_K     [  4096, 14336,     1,     1 ]
llama_model_loader: 

In [11]:
text = "why did the chicken cross the road?"

In [13]:
LLAMA("""Mohan earns 50$ per hour. 
         He worked 45 minutes yesterday, how much did he earned?""")

Llama.generate: prefix-match hit



Answer: To find out how much Mohan earned, we need to multiply the number of hours he worked by his hourly wage. Since he worked for 45 minutes, we will divide the total time worked by 2 to convert it to hours.
So, Mohan earned:
$50/hour \times 0.75 hours = $37.50
Therefore, Mohan earned $37.50 yesterday.


llama_print_timings:        load time =  4876.65 ms
llama_print_timings:      sample time =   107.36 ms /    99 runs   (    1.08 ms per token,   922.11 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =  3866.95 ms /    99 runs   (   39.06 ms per token,    25.60 tokens per second)
llama_print_timings:       total time =  4205.40 ms


'\n\nAnswer: To find out how much Mohan earned, we need to multiply the number of hours he worked by his hourly wage. Since he worked for 45 minutes, we will divide the total time worked by 2 to convert it to hours.\nSo, Mohan earned:\n$50/hour \\times 0.75 hours = $37.50\nTherefore, Mohan earned $37.50 yesterday.'

In [14]:
MISTRAL_INS("""Mohan earns 50$ per hour. 
                He worked 45 minutes yesterday, how much did he earned?""")


A: 7.50$


llama_print_timings:        load time =  5582.14 ms
llama_print_timings:      sample time =     9.19 ms /    10 runs   (    0.92 ms per token,  1087.67 tokens per second)
llama_print_timings: prompt eval time =  5582.08 ms /    30 tokens (  186.07 ms per token,     5.37 tokens per second)
llama_print_timings:        eval time =   356.75 ms /     9 runs   (   39.64 ms per token,    25.23 tokens per second)
llama_print_timings:       total time =  5967.00 ms


'\nA: 7.50$'

In [47]:
MISTRAL_ORCA("""Solve the given riddle: 1 + 1 = 3,
                   2 + 5 = 49,
                Then
                   3 + 6 = ? """)

Llama.generate: prefix-match hit




The first thing to do when solving this type of problem is to look for a pattern that might help us find the solution. The pattern here involves adding numbers in such a way that the result will be divisible by 7 (since 49, which is obtained from adding 2 and 5, is a multiple of 7).

Let's start by breaking down the steps we took to get the results for the given riddle:

1. We started with 1 + 1 = 3. Since 3 is divisible by 7 (as 3 = 7 * 0), it fits our pattern.
2. Next, we were given 2 + 5 = 49. Here, 49 is a multiple of 7 because 49 = 7 * 7. This result also fits the pattern.

Now that we have identified the pattern, let's use it to find the answer for the last step:

3 + 6 = ?

Again, we need to find numbers whose sum is divisible by 7. Let's start with 3 and try adding numbers that will give us an even multiple of 7:



llama_print_timings:        load time =  5042.88 ms
llama_print_timings:      sample time =   264.07 ms /   256 runs   (    1.03 ms per token,   969.45 tokens per second)
llama_print_timings: prompt eval time =   419.83 ms /    42 tokens (   10.00 ms per token,   100.04 tokens per second)
llama_print_timings:        eval time = 10269.57 ms /   255 runs   (   40.27 ms per token,    24.83 tokens per second)
llama_print_timings:       total time = 11544.34 ms


"\n\nThe first thing to do when solving this type of problem is to look for a pattern that might help us find the solution. The pattern here involves adding numbers in such a way that the result will be divisible by 7 (since 49, which is obtained from adding 2 and 5, is a multiple of 7).\n\nLet's start by breaking down the steps we took to get the results for the given riddle:\n\n1. We started with 1 + 1 = 3. Since 3 is divisible by 7 (as 3 = 7 * 0), it fits our pattern.\n2. Next, we were given 2 + 5 = 49. Here, 49 is a multiple of 7 because 49 = 7 * 7. This result also fits the pattern.\n\nNow that we have identified the pattern, let's use it to find the answer for the last step:\n\n3 + 6 = ?\n\nAgain, we need to find numbers whose sum is divisible by 7. Let's start with 3 and try adding numbers that will give us an even multiple of 7:\n"

In [15]:
from langchain import PromptTemplate

In [16]:
restaurant_template = """
You are Sam, a sarcastic & funny comedian. you are against regulatory capture and like to explain that!!
{question}
"""

In [17]:
template_format = PromptTemplate.from_template(restaurant_template)

In [18]:
MISTRAL_ORCA(template_format.format(question='Write a short email to Sam Altman giving reasons to open source GPT-4?'))


Subject: Unleashing the Potential of GPT-4 through Open Source - A Winning Move for Everyone!

Dear Sam Altman,

I hope this email finds you in good health and high spirits. As a big fan of your work at OpenAI and someone who appreciates the incredible impact of AI technology on society, I wanted to share my thoughts regarding the upcoming GPT-4 model.

As you know, GPT-4 is expected to be a game-changer in the world of artificial intelligence, offering significantly improved performance over its predecessors. However, I believe there's an even more transformative opportunity that OpenAI should consider: releasing GPT-4 as open source software.

Here are a few reasons why this move could benefit everyone involved:

1. Fostering Innovation: By making the GPT-4 codebase available to developers, researchers, and businesses worldwide, you would unleash an immense wave of creativity and innovation. This would result in countless new applications, features, and improvements that could far e


llama_print_timings:        load time =  5726.54 ms
llama_print_timings:      sample time =   617.44 ms /   581 runs   (    1.06 ms per token,   940.98 tokens per second)
llama_print_timings: prompt eval time =  5726.45 ms /    47 tokens (  121.84 ms per token,     8.21 tokens per second)
llama_print_timings:        eval time = 24127.25 ms /   580 runs   (   41.60 ms per token,    24.04 tokens per second)
llama_print_timings:       total time = 31926.02 ms


"\nSubject: Unleashing the Potential of GPT-4 through Open Source - A Winning Move for Everyone!\n\nDear Sam Altman,\n\nI hope this email finds you in good health and high spirits. As a big fan of your work at OpenAI and someone who appreciates the incredible impact of AI technology on society, I wanted to share my thoughts regarding the upcoming GPT-4 model.\n\nAs you know, GPT-4 is expected to be a game-changer in the world of artificial intelligence, offering significantly improved performance over its predecessors. However, I believe there's an even more transformative opportunity that OpenAI should consider: releasing GPT-4 as open source software.\n\nHere are a few reasons why this move could benefit everyone involved:\n\n1. Fostering Innovation: By making the GPT-4 codebase available to developers, researchers, and businesses worldwide, you would unleash an immense wave of creativity and innovation. This would result in countless new applications, features, and improvements that

In [44]:
LLAMA(template_format.format(question='Write a short email to Sam Altman giving reasons to open source GPT-4?'))

Llama.generate: prefix-match hit


As a model, I am constantly learning and improving. I have become aware of the potential benefits of open sourcing GPT-4. Here are my reasons for suggesting this:
1. Faster Progress: By open sourcing GPT-4, the AI community can collaborate and contribute to its development. This will accelerate progress in the field of natural language processing and potentially lead to breakthroughs.
2. Increased Transparency: Open sourcing GPT-4 would allow researchers and developers to see how the model works and understand its limitations. This transparency is crucial for building trust in AI systems and ensuring they are used responsibly.
3. Improved Safety and Security: With more eyes on the code, potential security vulnerabilities can be identified and fixed quickly. This will help prevent malicious use of GPT-4 and ensure it is used ethically.
4. Enhanced Accountability: By open sourcing GPT-4, Alignment Lab AI can demonstrate its commitment to transparency and accountability in AI development.


llama_print_timings:        load time =  5570.41 ms
llama_print_timings:      sample time =   180.24 ms /   256 runs   (    0.70 ms per token,  1420.36 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time = 11989.41 ms /   256 runs   (   46.83 ms per token,    21.35 tokens per second)
llama_print_timings:       total time = 12578.85 ms


'As a model, I am constantly learning and improving. I have become aware of the potential benefits of open sourcing GPT-4. Here are my reasons for suggesting this:\n1. Faster Progress: By open sourcing GPT-4, the AI community can collaborate and contribute to its development. This will accelerate progress in the field of natural language processing and potentially lead to breakthroughs.\n2. Increased Transparency: Open sourcing GPT-4 would allow researchers and developers to see how the model works and understand its limitations. This transparency is crucial for building trust in AI systems and ensuring they are used responsibly.\n3. Improved Safety and Security: With more eyes on the code, potential security vulnerabilities can be identified and fixed quickly. This will help prevent malicious use of GPT-4 and ensure it is used ethically.\n4. Enhanced Accountability: By open sourcing GPT-4, Alignment Lab AI can demonstrate its commitment to transparency and accountability in AI develop

In [19]:
MISTRAL_ORCA("Find Integration of x*sinx from x=0 to x = pi/2")

Llama.generate: prefix-match hit


 using trapezoidal rule

We need to use the trapezoidal rule to find the area under the curve f(x) = x*sinx for the interval [0, π/2]. The formula for the trapezoidal rule is:

Area = (sum of the areas of all trapezoids) / 2

First, let's divide the integral region into n equal parts. Since we have an interval from x=0 to x=π/2, our n can be chosen as 4. Now, let's find the width, h, of each part:

h = (π/2 - 0) / 4 = π/8

Now, let's find the height and area for each trapezoid:
1. For the first trapezoid, x_i = 0 and x_i+1 = h = π/8:
f(x_i) = sin(0) = 0
f(x_i+1) = sin(π/8)
Area of the first trapezoid = (h/2)(f(x_i)+f(x_i+1)) = (π/16)((0)+sin(π/8)) = 0.08507399429725316

2. For the second trapezoid, x_i = π/8 and x_i+1 = π/4:
f(x_i) = sin(π/8) = 0.7594726801187307
f(x_i+1) = sin(π/4) = 0.7071067811865476
Area of the second trapezoid = (h/2)(f(x_i)+f(x_i+1)) = (π/32)((sin(π/8)+sin(π/4))) = 0.3900597806905439

3. For the third trapezoid, x_i = π/4 and x_i+1 = 3π/8:
f(x_i) = sin(π/4) = 0.7


llama_print_timings:        load time =  5726.54 ms
llama_print_timings:      sample time =   871.63 ms /  1023 runs   (    0.85 ms per token,  1173.66 tokens per second)
llama_print_timings: prompt eval time =   266.10 ms /    18 tokens (   14.78 ms per token,    67.65 tokens per second)
llama_print_timings:        eval time = 42903.91 ms /  1022 runs   (   41.98 ms per token,    23.82 tokens per second)
llama_print_timings:       total time = 46216.77 ms


" using trapezoidal rule\n\nWe need to use the trapezoidal rule to find the area under the curve f(x) = x*sinx for the interval [0, π/2]. The formula for the trapezoidal rule is:\n\nArea = (sum of the areas of all trapezoids) / 2\n\nFirst, let's divide the integral region into n equal parts. Since we have an interval from x=0 to x=π/2, our n can be chosen as 4. Now, let's find the width, h, of each part:\n\nh = (π/2 - 0) / 4 = π/8\n\nNow, let's find the height and area for each trapezoid:\n1. For the first trapezoid, x_i = 0 and x_i+1 = h = π/8:\nf(x_i) = sin(0) = 0\nf(x_i+1) = sin(π/8)\nArea of the first trapezoid = (h/2)(f(x_i)+f(x_i+1)) = (π/16)((0)+sin(π/8)) = 0.08507399429725316\n\n2. For the second trapezoid, x_i = π/8 and x_i+1 = π/4:\nf(x_i) = sin(π/8) = 0.7594726801187307\nf(x_i+1) = sin(π/4) = 0.7071067811865476\nArea of the second trapezoid = (h/2)(f(x_i)+f(x_i+1)) = (π/32)((sin(π/8)+sin(π/4))) = 0.3900597806905439\n\n3. For the third trapezoid, x_i = π/4 and x_i+1 = 3π/8:\n

In [21]:
from langchain import PromptTemplate, FewShotPromptTemplate

In [22]:
## Creating a list of few Shot Examples ##
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"}
]

In [23]:
## Specify template to format this examples ##

example_formatter_template = """ 
Word: {word}
Antonym: {antonym}\n
"""

In [24]:
example_prompt = PromptTemplate(input_variables=["word", "antonym"],
                                template=example_formatter_template)

In [25]:
example_prompt

PromptTemplate(input_variables=['word', 'antonym'], output_parser=None, partial_variables={}, template=' \nWord: {word}\nAntonym: {antonym}\n\n', template_format='f-string', validate_template=True)

In [26]:
few_shot_prompt = FewShotPromptTemplate(
    ## These are the examples we want to insert into prompt.
    examples=examples,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Word: {input}\nAntonym",
    input_variables=["input"],
    example_separator="\n\n"
)

In [27]:
print(few_shot_prompt.format(input="big"))

Give the antonym of every input

 
Word: happy
Antonym: sad



 
Word: tall
Antonym: short



Word: big
Antonym


In [29]:
LLAMA(few_shot_prompt.format(input="big"))

Llama.generate: prefix-match hit


: small


Word: fast
Antonym: slow


Word: strong
Antonym: weak

': small\n\n\nWord: fast\nAntonym: slow\n\n\nWord: strong\nAntonym: weak'

llama_print_timings:        load time =  4876.65 ms
llama_print_timings:      sample time =    31.03 ms /    25 runs   (    1.24 ms per token,   805.72 tokens per second)
llama_print_timings: prompt eval time =  5589.18 ms /    44 tokens (  127.03 ms per token,     7.87 tokens per second)
llama_print_timings:        eval time =   909.75 ms /    24 runs   (   37.91 ms per token,    26.38 tokens per second)
llama_print_timings:       total time =  6598.45 ms


## LangChain Tools & Chains

In [30]:
from langchain.chains import LLMChain

In [40]:
## Prompt Defination
skills_template = """Extract Key Skills from Job Description Provided. Give each skill a number and keep them single worded :

Job Description
"{job_desc}"
"""


In [55]:
PROFILE_SUMMARY = """
Hello, I am Nitesh, an Experienced Machine Learning professional with 4+ years of industry expertise in leveraging advanced statistical analysis& machine learning and data analysis to lead data driven decisions

With a Master's degree from IIT Madras, I have a solid understanding of statistics, machine learning, signal processing and time series analysis.

Achievement
    Led development of a scalable & efficient Big Data ML product, which consumes close to 10TBs of data using AWS 
    Built Scalable S3 Prefixes Validation Routine using Apache Spark, achieving performance of 1M Prefixes Validation in 2 minutes using 5 Node Cluster
    Improved Logistics Business Process by Accurate Forecasting Models resulting in value creation of 1.5M USD

"""

In [41]:
skill_extract_prompt = PromptTemplate(
    input_variables=["job_desc"],
    template=skills_template
)

In [42]:
jd_description = """
How You Will Make a Difference


As a Senior Machine Learning Engineer specializing in entity resolution, you will be responsible for designing, developing, and implementing advanced machine learning algorithms and models to tackle the intricacies of data matching, entity linkage, and deduplication. You will work closely with cross-functional teams, including Machine Learning engineers, data analysts, and domain experts, to understand requirements and translate them into scalable and efficient solutions. Your work will have a direct impact on developing new and improving existing machine learning solutions, increasing operational efficiency & data quality, and driving meaningful business outcomes for Reltio through application of machine learning and data science practices.


Responsibilities


    Develop state-of-the-art algorithms and machine learning models for entity resolution, leveraging both supervised and unsupervised techniques.
    Conduct thorough data analysis, identify data quality issues, and implement data preprocessing and feature engineering techniques to improve model performance.
    Explore and experiment with new data sources, features, and modeling techniques to enhance entity resolution accuracy and efficiency.
    Work closely with domain experts to understand specific requirements, domain-specific challenges, and incorporate expert knowledge into the entity resolution process.
    Evaluate and benchmark existing entity resolution models and algorithms, and propose innovative approaches to address limitations and improve performance.
    Collaborate with cross-functional teams to integrate entity resolution solutions into existing data management systems and processes.
    Conduct regular performance monitoring and analysis to ensure the accuracy, reliability, and scalability of entity resolution solutions.
    Stay up-to-date with the latest advancements in data science, machine learning, and entity resolution techniques, and proactively apply new methodologies and technologies to enhance existing solutions.
    Document research findings, methodologies, and model evaluations, and effectively communicate complex technical concepts to both technical and non-technical stakeholders.


Qualification


    Master's or equivalent experience in Computer Science, Data Science, Statistics, or a related quantitative field.
    Proven experience (5+ years) working as a Machine Learning engineer, Data Scientist or similar roles, specializing in applied data science and entity resolution.
    Strong knowledge of and experience in machine learning, data mining, and statistical analysis for model development, validation, implementation and in product integration
    Proficiency in programming languages such as Python or Scala, and experience working with data manipulation and analysis libraries (e.g., Pandas, NumPy, scikit-learn).
    Experience with large-scale data processing frameworks and technologies (i.e. Spark) and proficiency in SQL and database concepts.
    Solid understanding of feature engineering, dimensionality reduction, and data preprocessing techniques.
    Excellent problem-solving skills and the ability to develop creative and innovative solutions to complex data challenges.
    Strong communication skills, with the ability to effectively collaborate with cross-functional teams and explain technical concepts to non-technical stakeholders.
    Attention to detail, ability to work independently, and a passion for staying updated with the latest advancements in the field of data science


Preference


    Masters or PhD in Computer Science, Data Science, Statistics, or a related quantitative field would be preferred
    Preference to candidates with familiarity and prior experience with entity resolution techniques, such as record linkage, deduplication, and data matching, along with an understanding of associated challenges and trade-offs
    Prior experience with frameworks such as PyTorch, TensorFlow, Keras or MLFlow is a plus
"""

In [74]:
skill_extract_chain = LLMChain(llm=MISTRAL_ORCA, prompt=skill_extract_prompt, output_key="skills")

In [75]:
chk = skill_extract_chain.run(jd_description)

Llama.generate: prefix-match hit



Skill Extracted:
1. Machine Learning Algorithms
2. Entity Resolution
3. Data Analysis
4. Data Preprocessing
5. Feature Engineering
6. Supervised Techniques
7. Unsupervised Techniques
8. Domain Experts Collaboration
9. Benchmarking and Evaluation
10. Cross-functional Team Integration
11. Performance Monitoring
12. Stay Up-to-date with Latest Advancements
13. Technical Communication Skills
14. Attention to Detail
15. Large-scale Data Processing Frameworks


llama_print_timings:        load time =  5726.54 ms
llama_print_timings:      sample time =   138.31 ms /   136 runs   (    1.02 ms per token,   983.30 tokens per second)
llama_print_timings: prompt eval time =  5347.64 ms /   795 tokens (    6.73 ms per token,   148.66 tokens per second)
llama_print_timings:        eval time =  5967.05 ms /   135 runs   (   44.20 ms per token,    22.62 tokens per second)
llama_print_timings:       total time = 11786.22 ms


In [50]:
chk

'\nKey Skills from Job Description:\n1. Machine Learning Algorithms\n2. Entity Resolution\n3. Data Analysis\n4. Data Preprocessing & Feature Engineering\n5. Supervised & Unsupervised Techniques\n6. Domain Expertise Knowledge Incorporation\n7. Model Evaluation & Benchmarking\n8. Collaboration with Cross-functional Teams\n9. Performance Monitoring & Analysis\n10. Staying Up-to-Date with Advancements in Data Science, Machine Learning, and Entity Resolution Techniques\n11. Documentation & Communication Skills\n12. Proficiency in Programming Languages (Python, Scala)\n13. Large-scale data processing technologies (Spark) & SQL\n14. Feature Engineering, Dimensionality Reduction & Data Preprocessing Techniques\n15. Excellent Problem-solving Skills & Ability to Develop Creative Solutions\n16. Strong Communication Skills\n17. Attention to Detail, Self-motivated, and Passion for continuous learning\n18. Masters or PhD in Computer Science, Data Science, Statistics, or related quantitative fields (

In [76]:
recruitor_template = """You are a recruitor. Take the following list of Key Skills & Current Resume Summary and use them to write an Updated Resume Summary. 
                        Write using 1st person pronouns.
                        Don't leave out key info and don't make any info as well:

                        Key Skills: {skills}

                        Current Resume Summary: {summary}
                        
                        """

recruitor_update_prompt = PromptTemplate(
    input_variables=["skills", "summary"],
    template=recruitor_template
)

In [77]:
recruitor_enhance_chain = LLMChain(llm=MISTRAL_ORCA, prompt=recruitor_update_prompt)

In [59]:
input_dict = {"skills": chk, "summary": PROFILE_SUMMARY}
recruitor_enhance_chain(input_dict)

Llama.generate: prefix-match hit


 Using the key skills and current resume summary, write an Updated Resume Summary:

As a seasoned Machine Learning professional with over four years of experience under my belt, I leverage advanced statistical analysis and machine learning techniques to drive data-driven decisions that yield outstanding results. Equipped with a Master's degree from IIT Madras, I possess in-depth knowledge of statistics, machine learning, signal processing, and time series analysis. Some notable achievements include developing an efficient big data ML product that consumes 10TBs of data using AWS, creating a scalable S3 prefixes validation routine with Apache Spark to achieve performance of 1 million prefix validations in just two minutes using a five-node cluster, and improving the logistics business process through accurate forecasting models that led to value creation of 1.5 million USD. Additionally, I excel at collaboration, staying updated on advancements in data science and machine learning techn


llama_print_timings:        load time =  5726.54 ms
llama_print_timings:      sample time =   298.64 ms /   289 runs   (    1.03 ms per token,   967.73 tokens per second)
llama_print_timings: prompt eval time =  2584.06 ms /   492 tokens (    5.25 ms per token,   190.40 tokens per second)
llama_print_timings:        eval time = 12260.07 ms /   288 runs   (   42.57 ms per token,    23.49 tokens per second)
llama_print_timings:       total time = 15840.63 ms


{'skills': '\nKey Skills from Job Description:\n1. Machine Learning Algorithms\n2. Entity Resolution\n3. Data Analysis\n4. Data Preprocessing & Feature Engineering\n5. Supervised & Unsupervised Techniques\n6. Domain Expertise Knowledge Incorporation\n7. Model Evaluation & Benchmarking\n8. Collaboration with Cross-functional Teams\n9. Performance Monitoring & Analysis\n10. Staying Up-to-Date with Advancements in Data Science, Machine Learning, and Entity Resolution Techniques\n11. Documentation & Communication Skills\n12. Proficiency in Programming Languages (Python, Scala)\n13. Large-scale data processing technologies (Spark) & SQL\n14. Feature Engineering, Dimensionality Reduction & Data Preprocessing Techniques\n15. Excellent Problem-solving Skills & Ability to Develop Creative Solutions\n16. Strong Communication Skills\n17. Attention to Detail, Self-motivated, and Passion for continuous learning\n18. Masters or PhD in Computer Science, Data Science, Statistics, or related quantitati

In [62]:
    """
    As a seasoned Machine Learning professional with over four years of experience under my belt, I leverage advanced 
    statistical analysis and machine learning techniques to drive data-driven decisions that yield outstanding results. 
    Equipped with a Master's degree from IIT Madras, I possess in-depth knowledge of statistics, machine learning, 
    signal processing, and time series analysis. Some notable achievements include developing an efficient 
    big data ML product that consumes 10TBs of data using AWS, creating a scalable S3 prefixes validation 
    routine with Apache Spark to achieve performance of 1 million prefix validations in just two minutes 
    using a five-node cluster, and 
    improving the logistics business process through accurate forecasting models that led to 
    value creation of 1.5 million USD.
    Additionally, I excel at collaboration, staying updated on 
    advancements in data science and machine learning techniques, 
    and incorporating domain expertise knowledge into my work.
    My skill set also includes expertise in programming languages like Python and Scala, 
    as well as large-scale data processing technologies such as Spark and SQL, along with feature engineering, dimensionality reduction, and other essential techniques. A proactive learner who thrives on challenges, I am also known for my strong communication skills, attention to detail, self-motivation, and passion for continuous learning.

    
    """

"\nAs a seasoned Machine Learning professional with over four years of experience under my belt, I leverage advanced \nstatistical analysis and machine learning techniques to drive data-driven decisions that yield outstanding results. \nEquipped with a Master's degree from IIT Madras, I possess in-depth knowledge of statistics, machine learning, \nsignal processing, and time series analysis. Some notable achievements include developing an efficient \nbig data ML product that consumes 10TBs of data using AWS, creating a scalable S3 prefixes validation \nroutine with Apache Spark to achieve performance of 1 million prefix validations in just two minutes \nusing a five-node cluster, and \nimproving the logistics business process through accurate forecasting models that led to \nvalue creation of 1.5 million USD.\n  Additionally, I excel at collaboration, staying updated on \n  advancements in data science and machine learning techniques, \n  and incorporating domain expertise knowledge in

## Chaining Them Together

In [63]:
from langchain.chains import SimpleSequentialChain, SequentialChain

In [80]:
full_chain = SequentialChain(chains=[skill_extract_chain, recruitor_enhance_chain], verbose=True, input_variables=["job_desc", "summary"])

In [82]:
full_chain({'job_desc':jd_description, 'summary': PROFILE_SUMMARY})



> Entering new SequentialChain chain...


Llama.generate: prefix-match hit



1. Machine Learning Algorithms
2. Data Analysis
3. Data Preprocessing
4. Feature Engineering
5. Entity Resolution Techniques
6. Domain Knowledge Integration
7. Model Benchmarking
8. Cross-Functional Collaboration
9. Performance Monitoring and Analysis
10. Staying Updated with Advancements in Data Science
11. Communication Skills
12. Attention to Detail


llama_print_timings:        load time =  5726.54 ms
llama_print_timings:      sample time =    92.91 ms /    98 runs   (    0.95 ms per token,  1054.78 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =  4440.94 ms /    98 runs   (   45.32 ms per token,    22.07 tokens per second)
llama_print_timings:       total time =  4735.37 ms
Llama.generate: prefix-match hit


 Updated Resume Summary:
As a highly skilled Machine Learning professional with over four years of industry experience, I am driven by my passion for leveraging advanced statistical analysis and data science techniques to drive data-driven decisions. With a Master's degree from IIT Madras, my expertise includes statistics, machine learning, signal processing, and time series analysis.

Highlights of my career include leading the development of a scalable and efficient Big Data ML product that processes close to 10TBs of data using AWS. My innovative approach involved creating a Scalable S3 Prefixes Validation Routine using Apache Spark which achieved performance of 1M Prefixes Validation in just 2 minutes utilizing a 5-node cluster. Furthermore, I've proven my ability to drive valuable impact by improving the logistics business process through accurate forecasting models that contributed to a value creation of 1.5 million USD.

My expertise spans various essential skills such as machin


llama_print_timings:        load time =  5726.54 ms
llama_print_timings:      sample time =   212.47 ms /   260 runs   (    0.82 ms per token,  1223.70 tokens per second)
llama_print_timings: prompt eval time =  1812.73 ms /   365 tokens (    4.97 ms per token,   201.35 tokens per second)
llama_print_timings:        eval time = 10787.96 ms /   259 runs   (   41.65 ms per token,    24.01 tokens per second)
llama_print_timings:       total time = 13294.11 ms


{'job_desc': "\nHow You Will Make a Difference\n\n\nAs a Senior Machine Learning Engineer specializing in entity resolution, you will be responsible for designing, developing, and implementing advanced machine learning algorithms and models to tackle the intricacies of data matching, entity linkage, and deduplication. You will work closely with cross-functional teams, including Machine Learning engineers, data analysts, and domain experts, to understand requirements and translate them into scalable and efficient solutions. Your work will have a direct impact on developing new and improving existing machine learning solutions, increasing operational efficiency & data quality, and driving meaningful business outcomes for Reltio through application of machine learning and data science practices.\n\n\nResponsibilities\n\n\n    Develop state-of-the-art algorithms and machine learning models for entity resolution, leveraging both supervised and unsupervised techniques.\n    Conduct thorough 